# [모듈 2.1] CodeBuild 프로젝트 생성

- 개발자 가이드
    - [Create a build project (AWS CLI)](https://docs.aws.amazon.com/codebuild/latest/userguide/create-project-cli.html)
    

aws codebuild list-projects --sort-by NAME --sort-order ASCENDING    
aws codebuild batch-get-projects --names iris-build-project    

- 개발자 가이드
    - [Create a build project (AWS CLI)](https://docs.aws.amazon.com/codebuild/latest/userguide/create-project-cli.html)
    

aws codebuild list-projects --sort-by NAME --sort-order ASCENDING    
aws codebuild batch-get-projects --names iris-build-project    

# 1. 환경 설정

## 1.0 기존의 변수 불러오기

In [162]:
%store -r repository_name
%store -r s3_input_data_uri

## 1.1. 파라미터 설정

In [163]:
import sagemaker
import boto3

region = sagemaker.Session().boto_region_name

account_id = boto3.client('sts').get_caller_identity().get('Account')

project_prefix = "Train-NCF"
code_build_project_name = "ncf-training-sm-pipeline"
sm_pipeline_name = "ncf-sm-pipeline"
model_package_group_name = "NCF-Model-CodePipeline"
inference_image_uri = "763104351884.dkr.ecr.us-east-1.amazonaws.com/pytorch-inference:1.8.1-gpu-py3"

print("region: ", region)
print("account_id: ", account_id)
print("project_prefix: ", project_prefix)
print("code_build_project_name: ", code_build_project_name)
print("sm_pipeline_name: ", sm_pipeline_name)
print("model_package_group_name: ", model_package_group_name)
print("inference_image_uri: ", inference_image_uri)

region:  us-east-1
account_id:  057716757052
project_prefix:  Train-NCF
code_build_project_name:  ncf-training-sm-pipeline
sm_pipeline_name:  ncf-sm-pipeline
model_package_group_name:  NCF-Model-CodePipeline
inference_image_uri:  763104351884.dkr.ecr.us-east-1.amazonaws.com/pytorch-inference:1.8.1-gpu-py3


In [164]:
isDefaultBucket = True

if isDefaultBucket:

    bucket = sagemaker.Session().default_bucket()
else:
    bucket = "<Type your bucket name>"
    
print("Bucket: ", bucket)    


Bucket:  sagemaker-us-east-1-057716757052


## 1.2. 역할 생성 및 정의
### 아래 Code Build 역할 ARN 을 넣어 주세요
- build_service_role: 은 아래와 미리 생성이 됨
    -  "arn:aws:iam::XXXXXX:role/codebuild-gsmoon"

In [165]:
# build_service_role = "arn:aws:iam::XXXXXX:role/codebuild-gsmoon"
build_service_role = "arn:aws:iam::057716757052:role/codebuild-gsmoon"

# 2. 빌드 프로젝트  JSON 정의
- CodeBuild serviceRole 과 SAGEMAKER_PIPELINE_ROLE_ARN 을 동일한 Role 로 사용함.

```
# SAGEMAKER_PIPELINE_ROLE_ARN
"name": "SAGEMAKER_PIPELINE_ROLE_ARN",
"value": build_service_role,
```

```
# CodeBuild serviceRole
  "serviceRole": build_service_role,  
```

In [166]:
build_dic = {
  "name": code_build_project_name,
  "description": "Hello Build Project",
  "source": {
    "type": "CODEPIPELINE",
    "buildspec": "codebuild-buildspec.yml",
  },
  "artifacts": {
    "type": "CODEPIPELINE",
    "name": code_build_project_name,
  },
  "cache": {
    "type": "NO_CACHE",
  },
  "environment": {
            "type": "LINUX_CONTAINER",
            "image": "aws/codebuild/amazonlinux2-x86_64-standard:3.0",
            "computeType": "BUILD_GENERAL1_SMALL",
            "environmentVariables": [
                {
                    "name": "IMAGE_REPO_NAME",
                    "value": "iris-model",
                    "type": "PLAINTEXT"
                },
                {
                    "name": "IMAGE_TAG",
                    "value": "latest",
                    "type": "PLAINTEXT"
                },
                {
                    "name": "AWS_ACCOUNT_ID",
                    "value": account_id,
                    "type": "PLAINTEXT"
                },
                {
                    "name": "SAGEMAKER_PIPELINE_ROLE_ARN",
                    "value": build_service_role,
                    "type": "PLAINTEXT"
                },        
                {
                    "name": "SAGEMAKER_PIPELINE_NAME",
                    "value": sm_pipeline_name,
                    "type": "PLAINTEXT"
                },                        
                {
                    "name": "AWS_DEFAULT_REGION",
                    "value": region,
                    "type": "PLAINTEXT"
                },
                {
                    "name": "TEMPLATE_BUCKET",
                    "value": bucket,
                    "type": "PLAINTEXT"
                },
                {
                    "name": "s3_input_data_uri",
                    "value": s3_input_data_uri,
                    "type": "PLAINTEXT"
                },
                {
                    "name": "code_repository_name",
                    "value": repository_name,
                    "type": "PLAINTEXT"
                },                
                {
                    "name": "project_prefix",
                    "value": project_prefix,
                    "type": "PLAINTEXT"
                },
                {
                    "name": "model_package_group_name",
                    "value": model_package_group_name,
                    "type": "PLAINTEXT"
                },                                
                {
                    "name": "inference_image_uri",
                    "value": inference_image_uri,
                    "type": "PLAINTEXT"
                },                
                {
                    "name": "TEMPLATE_PREFIX",
                    "value": "codebuild",
                    "type": "PLAINTEXT"
                }
                
            ],
            "privilegedMode": False,
            "imagePullCredentialsType": "CODEBUILD"
        },
  "serviceRole": build_service_role,
  "timeoutInMinutes": 60,
  "queuedTimeoutInMinutes": 480,
  "badgeEnabled": False,
  "logsConfig": {
    "cloudWatchLogs": {
      "status": "ENABLED",
    },
    "s3Logs": {
      "status": "DISABLED",
      "encryptionDisabled": False
    }
  },
}

## 2.1. Dic 포맷을 JSON 으로 변경

In [167]:
import json

build_json = json.dumps(build_dic)
json.loads(build_json)

{'name': 'ncf-training-sm-pipeline',
 'description': 'Hello Build Project',
 'source': {'type': 'CODEPIPELINE', 'buildspec': 'codebuild-buildspec.yml'},
 'artifacts': {'type': 'CODEPIPELINE', 'name': 'ncf-training-sm-pipeline'},
 'cache': {'type': 'NO_CACHE'},
 'environment': {'type': 'LINUX_CONTAINER',
  'image': 'aws/codebuild/amazonlinux2-x86_64-standard:3.0',
  'computeType': 'BUILD_GENERAL1_SMALL',
  'environmentVariables': [{'name': 'IMAGE_REPO_NAME',
    'value': 'iris-model',
    'type': 'PLAINTEXT'},
   {'name': 'IMAGE_TAG', 'value': 'latest', 'type': 'PLAINTEXT'},
   {'name': 'AWS_ACCOUNT_ID', 'value': '057716757052', 'type': 'PLAINTEXT'},
   {'name': 'SAGEMAKER_PIPELINE_ROLE_ARN',
    'value': 'arn:aws:iam::057716757052:role/codebuild-gsmoon',
    'type': 'PLAINTEXT'},
   {'name': 'SAGEMAKER_PIPELINE_NAME',
    'value': 'ncf-sm-pipeline',
    'type': 'PLAINTEXT'},
   {'name': 'AWS_DEFAULT_REGION', 'value': 'us-east-1', 'type': 'PLAINTEXT'},
   {'name': 'TEMPLATE_BUCKET',
   

## 2.2. JSON 파일 저장

In [168]:
json_file_path = 'src/hello_build.json'
with open(json_file_path, "w") as outfile:
    outfile.write(build_json)

In [169]:
with open(json_file_path) as json_file:
    json_data = json.load(json_file)
    
json_data    

{'name': 'ncf-training-sm-pipeline',
 'description': 'Hello Build Project',
 'source': {'type': 'CODEPIPELINE', 'buildspec': 'codebuild-buildspec.yml'},
 'artifacts': {'type': 'CODEPIPELINE', 'name': 'ncf-training-sm-pipeline'},
 'cache': {'type': 'NO_CACHE'},
 'environment': {'type': 'LINUX_CONTAINER',
  'image': 'aws/codebuild/amazonlinux2-x86_64-standard:3.0',
  'computeType': 'BUILD_GENERAL1_SMALL',
  'environmentVariables': [{'name': 'IMAGE_REPO_NAME',
    'value': 'iris-model',
    'type': 'PLAINTEXT'},
   {'name': 'IMAGE_TAG', 'value': 'latest', 'type': 'PLAINTEXT'},
   {'name': 'AWS_ACCOUNT_ID', 'value': '057716757052', 'type': 'PLAINTEXT'},
   {'name': 'SAGEMAKER_PIPELINE_ROLE_ARN',
    'value': 'arn:aws:iam::057716757052:role/codebuild-gsmoon',
    'type': 'PLAINTEXT'},
   {'name': 'SAGEMAKER_PIPELINE_NAME',
    'value': 'ncf-sm-pipeline',
    'type': 'PLAINTEXT'},
   {'name': 'AWS_DEFAULT_REGION', 'value': 'us-east-1', 'type': 'PLAINTEXT'},
   {'name': 'TEMPLATE_BUCKET',
   

# 3. 빌드 프로젝트 생성

In [170]:
%%sh -s {json_file_path}
json_file_path=$1
echo $json_file_path
aws codebuild create-project --cli-input-json file://$json_file_path

src/hello_build.json
{
    "project": {
        "name": "ncf-training-sm-pipeline",
        "arn": "arn:aws:codebuild:us-east-1:057716757052:project/ncf-training-sm-pipeline",
        "description": "Hello Build Project",
        "source": {
            "type": "CODEPIPELINE",
            "buildspec": "codebuild-buildspec.yml",
            "insecureSsl": false
        },
        "artifacts": {
            "type": "CODEPIPELINE",
            "name": "ncf-training-sm-pipeline",
            "packaging": "NONE",
            "encryptionDisabled": false
        },
        "cache": {
            "type": "NO_CACHE"
        },
        "environment": {
            "type": "LINUX_CONTAINER",
            "image": "aws/codebuild/amazonlinux2-x86_64-standard:3.0",
            "computeType": "BUILD_GENERAL1_SMALL",
            "environmentVariables": [
                {
                    "name": "IMAGE_REPO_NAME",
                    "value": "iris-model",
                    "type": "PLAINTEXT"
  

# 4. 프로젝트 이름 저장

In [171]:
%store code_build_project_name
%store sm_pipeline_name
%store account_id
%store region
%store bucket


Stored 'code_build_project_name' (str)
Stored 'sm_pipeline_name' (str)
Stored 'account_id' (str)
Stored 'region' (str)
Stored 'bucket' (str)
